## CLEANING BASE IMMIGRATION DATA

In [162]:
import pandas as pd

immigration_df = pd.read_csv("Immigration_data.csv")
print(immigration_df.head())

  REF_DATE                   GEO           DGUID  \
0  1946-01                Canada  2016A000011124   
1  1946-01  Prince Edward Island     2016A000211   
2  1946-01           Nova Scotia     2016A000212   
3  1946-01         New Brunswick     2016A000213   
4  1946-01                Quebec     2016A000224   

  Components of population growth      UOM  UOM_ID SCALAR_FACTOR  SCALAR_ID  \
0                      Immigrants  Persons     249         units          0   
1                      Immigrants  Persons     249         units          0   
2                      Immigrants  Persons     249         units          0   
3                      Immigrants  Persons     249         units          0   
4                      Immigrants  Persons     249         units          0   

      VECTOR  COORDINATE    VALUE STATUS  SYMBOL  TERMINATED  DECIMALS  
0  v29850342         1.1  12738.0    NaN     NaN         NaN         0  
1  v29850352         3.1     73.0    NaN     NaN         NaN      

In [163]:
# Explore values in columns with ambiguous meaning
UOM_Unique = immigration_df['UOM'].unique()
Components_of_population_growth_Unique = immigration_df['Components of population growth'].unique()
status_unique = immigration_df['STATUS'].unique()
scalar_factor_unique = immigration_df['SCALAR_FACTOR'].unique()


print(f"list of unique UOM values", sorted(UOM_Unique)) # Eliminate Column
print(f"list of unique Components_of_population_growth_Unique values", sorted(Components_of_population_growth_Unique)) # Keep Column
print(f"list of unique STATUS values", (status_unique)) # Eliminate Column
print(f"list of unique SCALAR_FACTOR values", (scalar_factor_unique)) # Eliminate Column 

list of unique UOM values ['Persons']
list of unique Components_of_population_growth_Unique values ['Emigrants', 'Immigrants', 'Net emigration', 'Net non-permanent residents', 'Net temporary emigration', 'Non-permanent residents, inflows', 'Non-permanent residents, outflows', 'Returning emigrants']
list of unique STATUS values [nan '..']
list of unique SCALAR_FACTOR values ['units']


In [164]:
# The columns identified below are not necessary for our analysis and will be dropped from the dataframe
immigration_df_dropped = immigration_df.drop(columns=['DGUID','UOM','UOM_ID','SCALAR_FACTOR','SCALAR_ID','VECTOR','COORDINATE','STATUS','SYMBOL','TERMINATED','DECIMALS'])
immigration_df = immigration_df_dropped.rename(columns={'GEO':'province','Components of population growth':'immigration_category','VALUE':'num_people'})

In [165]:
# Create provincial dataframe by eliminating results for Canada
prov_immigration_df = immigration_df[immigration_df.province !='Canada']
print(prov_immigration_df.head())

# Check how many records are in the dataset
len(prov_immigration_df.index)

  REF_DATE              province immigration_category  num_people
1  1946-01  Prince Edward Island           Immigrants        73.0
2  1946-01           Nova Scotia           Immigrants       701.0
3  1946-01         New Brunswick           Immigrants       520.0
4  1946-01                Quebec           Immigrants      2305.0
5  1946-01               Ontario           Immigrants      5319.0


19607

In [166]:
# Keep only the rows with 'Immigrants','Net emigration', or 'Net non-permanent residents' in the immigration_category column. The values in this column are captured in 
# these 3 aggregations
prov_immigration_df = prov_immigration_df.loc[prov_immigration_df['immigration_category'].isin(['Immigrants','Net emigration','Net non-permanent residents'])]

# Check how many records are in the dataset after clean
len(prov_immigration_df.index)

9200

In [167]:
# Change Ref Date to reflect years only to prepare for year-by-year aggregation
prov_immigration_df["REF_DATE"] = pd.to_datetime(prov_immigration_df["REF_DATE"]).dt.strftime('%Y')

print(prov_immigration_df)

      REF_DATE               province         immigration_category  num_people
1         1946   Prince Edward Island                   Immigrants        73.0
2         1946            Nova Scotia                   Immigrants       701.0
3         1946          New Brunswick                   Immigrants       520.0
4         1946                 Quebec                   Immigrants      2305.0
5         1946                Ontario                   Immigrants      5319.0
...        ...                    ...                          ...         ...
21148     2024  Northwest Territories               Net emigration        -1.0
21151     2024  Northwest Territories  Net non-permanent residents        94.0
21154     2024                Nunavut                   Immigrants         9.0
21155     2024                Nunavut               Net emigration         0.0
21158     2024                Nunavut  Net non-permanent residents        12.0

[9200 rows x 4 columns]


In [168]:
# Group by 'REF_DATE','province',and 'immigration_category' and aggregate num people values
prov_immigration_df = prov_immigration_df.groupby(['REF_DATE','province','immigration_category']).agg({'num_people':'sum'})
print(prov_immigration_df)

                                                                              num_people
REF_DATE province                                immigration_category                   
1946     Alberta                                 Immigrants                       5771.0
         British Columbia                        Immigrants                       8619.0
         Manitoba                                Immigrants                       4615.0
         New Brunswick                           Immigrants                       3465.0
         Northwest Territories including Nunavut Immigrants                         21.0
...                                                                                  ...
2024     Saskatchewan                            Net emigration                    288.0
                                                 Net non-permanent residents      1033.0
         Yukon                                   Immigrants                        297.0
                     

In [169]:
# Rename Axis and reset index to flatten table out
prov_immigration_df = prov_immigration_df.rename_axis(columns=None).reset_index()
print(prov_immigration_df)

     REF_DATE                                 province  \
0        1946                                  Alberta   
1        1946                         British Columbia   
2        1946                                 Manitoba   
3        1946                            New Brunswick   
4        1946  Northwest Territories including Nunavut   
...       ...                                      ...   
2341     2024                             Saskatchewan   
2342     2024                             Saskatchewan   
2343     2024                                    Yukon   
2344     2024                                    Yukon   
2345     2024                                    Yukon   

             immigration_category  num_people  
0                      Immigrants      5771.0  
1                      Immigrants      8619.0  
2                      Immigrants      4615.0  
3                      Immigrants      3465.0  
4                      Immigrants        21.0  
...            

In [170]:
# Pivot the immigration_category column to create a separate column for each value there with its corresponding num_people value underneath
prov_immigration_df = prov_immigration_df.pivot(index=['REF_DATE', 'province'], columns='immigration_category',values='num_people')
print(prov_immigration_df)

immigration_category                              Immigrants  Net emigration  \
REF_DATE province                                                              
1946     Alberta                                      5771.0             NaN   
         British Columbia                             8619.0             NaN   
         Manitoba                                     4615.0             NaN   
         New Brunswick                                3465.0             NaN   
         Northwest Territories including Nunavut        21.0             NaN   
...                                                      ...             ...   
2024     Ontario                                     52781.0          5783.0   
         Prince Edward Island                         1330.0            24.0   
         Quebec                                      13369.0          1544.0   
         Saskatchewan                                 5611.0           288.0   
         Yukon                          

In [171]:
# Rename Axis and reset index to flatten table out
prov_immigration_df = prov_immigration_df.rename_axis(columns=None).reset_index()
print(prov_immigration_df)

    REF_DATE                                 province  Immigrants  \
0       1946                                  Alberta      5771.0   
1       1946                         British Columbia      8619.0   
2       1946                                 Manitoba      4615.0   
3       1946                            New Brunswick      3465.0   
4       1946  Northwest Territories including Nunavut        21.0   
..       ...                                      ...         ...   
975     2024                                  Ontario     52781.0   
976     2024                     Prince Edward Island      1330.0   
977     2024                                   Quebec     13369.0   
978     2024                             Saskatchewan      5611.0   
979     2024                                    Yukon       297.0   

     Net emigration  Net non-permanent residents  
0               NaN                          NaN  
1               NaN                          NaN  
2               Na

## CLEAN WAGES DATA

In [172]:
# Import wage data

wage_df = pd.read_csv("Wage_Data.csv")
print(wage_df)

         REF_DATE      GEO           DGUID  \
0         2001-01   Canada  2021A000011124   
1         2001-01   Canada  2021A000011124   
2         2001-01   Canada  2021A000011124   
3         2001-01   Canada  2021A000011124   
4         2001-01   Canada  2021A000011124   
...           ...      ...             ...   
23037583  2024-03  Nunavut     2021A000262   
23037584  2024-03  Nunavut     2021A000262   
23037585  2024-03  Nunavut     2021A000262   
23037586  2024-03  Nunavut     2021A000262   
23037587  2024-03  Nunavut     2021A000262   

                                                   Estimate  \
0                              Employment for all employees   
1                              Employment for all employees   
2                              Employment for all employees   
3                              Employment for all employees   
4                              Employment for all employees   
...                                                     ...   
230375

In [173]:
# Explore values in columns with ambiguous meaning
estimate_Unique = wage_df['Estimate'].unique()
NAICS_Unique = wage_df['North American Industry Classification System (NAICS)'].unique()
UOM_unique = wage_df['UOM'].unique()
status_unique = wage_df['STATUS'].unique()


print(f"list of unique estimate", sorted(estimate_Unique)) # Eliminate Column
print(f"list of unique NAICS", sorted(NAICS_Unique)) # Keep Column
print(f"list of unique UOM values", (UOM_unique)) # Eliminate Column
print(f"list of unique STATUS values", (status_unique)) # Eliminate Column 

list of unique estimate ['Average weekly earnings including overtime for all employees', 'Employment for all employees']
list of unique NAICS ['Accommodation and food services [72]', 'Administrative and support, waste management and remediation services [56]', 'Arts, entertainment and recreation [71]', 'Construction [23]', 'Durable goods [321N]', 'Educational services [61]', 'Finance and insurance [52]', 'Forestry, logging and support [11N]', 'Goods producing industries [11-33N]', 'Health care and social assistance [62]', 'Industrial aggregate excluding unclassified businesses [11-91N]', 'Industrial aggregate including unclassified businesses [00-91N]', 'Information and cultural industries [51]', 'Management of companies and enterprises [55]', 'Manufacturing [31-33]', 'Mining, quarrying, and oil and gas extraction [21]', 'Non-durable goods [311N]', 'Other services (except public administration) [81]', 'Professional, scientific and technical services [54]', 'Public administration [91]',

In [174]:
# We are only interested in the distinct provinces so we don't need to see canadian values
wage_df = wage_df[wage_df.GEO !='Canada']

# We are only interested in weekly earnings so we don't need to see aggregated earings for all employees
wage_df = wage_df[wage_df.Estimate !='Employment for all employees']

In [175]:
# The columns identified below are not necessary for our analysis and will be dropped from the dataframe
wage_df = wage_df.drop(columns=['DGUID','Estimate','UOM','UOM_ID','SCALAR_FACTOR','SCALAR_ID','VECTOR','COORDINATE','STATUS','SYMBOL','TERMINATED','DECIMALS','Release','North American Industry Classification System (NAICS)'])
wage_df = wage_df.rename(columns={'GEO':'province','Components of population growth':'immigration_category','VALUE':'weekly_wage'})
print(wage_df)

         REF_DATE                   province  weekly_wage
8895      2001-01  Newfoundland and Labrador          NaN
8896      2001-01  Newfoundland and Labrador          NaN
8897      2001-01  Newfoundland and Labrador          NaN
8898      2001-01  Newfoundland and Labrador          NaN
8899      2001-01  Newfoundland and Labrador          NaN
...           ...                        ...          ...
23037583  2024-03                    Nunavut          NaN
23037584  2024-03                    Nunavut          NaN
23037585  2024-03                    Nunavut          NaN
23037586  2024-03                    Nunavut          NaN
23037587  2024-03                    Nunavut          NaN

[10521927 rows x 3 columns]


In [176]:
# Change Ref Date to reflect years only to prepare for year-by-year aggregation
wage_df["REF_DATE"] = pd.to_datetime(wage_df["REF_DATE"]).dt.strftime('%Y')
print(wage_df.head(50))

     REF_DATE                   province  weekly_wage
8895     2001  Newfoundland and Labrador          NaN
8896     2001  Newfoundland and Labrador          NaN
8897     2001  Newfoundland and Labrador          NaN
8898     2001  Newfoundland and Labrador          NaN
8899     2001  Newfoundland and Labrador          NaN
8900     2001  Newfoundland and Labrador          NaN
8901     2001  Newfoundland and Labrador          NaN
8902     2001  Newfoundland and Labrador          NaN
8903     2001  Newfoundland and Labrador          NaN
8904     2001  Newfoundland and Labrador          NaN
8905     2001  Newfoundland and Labrador          NaN
8906     2001  Newfoundland and Labrador          NaN
8907     2001  Newfoundland and Labrador          NaN
8908     2001  Newfoundland and Labrador          NaN
8909     2001  Newfoundland and Labrador          NaN
8910     2001  Newfoundland and Labrador       593.60
8911     2001  Newfoundland and Labrador       593.60
8912     2001  Newfoundland 

In [177]:
# Group by 'REF_DATE','province',and 'immigration_category' and aggregate num people values
wage_df = wage_df.groupby(['REF_DATE','province']).agg({'weekly_wage':'mean'})
print(wage_df)

                                    weekly_wage
REF_DATE province                              
2001     Alberta                     714.613068
         British Columbia            717.266610
         Manitoba                    637.593449
         New Brunswick               625.547939
         Newfoundland and Labrador   634.622101
...                                         ...
2024     Ontario                    1357.052778
         Prince Edward Island       1064.517750
         Quebec                     1278.014931
         Saskatchewan               1245.763939
         Yukon                      1346.276220

[312 rows x 1 columns]


In [178]:
# Rename Axis and reset index to flatten table out
wage_df = wage_df.rename_axis(columns=None).reset_index()
print(wage_df.head(50))

   REF_DATE                   province  weekly_wage
0      2001                    Alberta   714.613068
1      2001           British Columbia   717.266610
2      2001                   Manitoba   637.593449
3      2001              New Brunswick   625.547939
4      2001  Newfoundland and Labrador   634.622101
5      2001      Northwest Territories   784.901499
6      2001                Nova Scotia   584.924018
7      2001                    Nunavut   690.123102
8      2001                    Ontario   741.539837
9      2001       Prince Edward Island   537.194820
10     2001                     Quebec   667.158953
11     2001               Saskatchewan   648.096357
12     2001                      Yukon   706.977970
13     2002                    Alberta   745.154624
14     2002           British Columbia   736.926552
15     2002                   Manitoba   656.486334
16     2002              New Brunswick   645.799215
17     2002  Newfoundland and Labrador   661.573347
18     2002 

## MERGE DATAFRAMES INTO TABLE READY FOR MODELLING

In [179]:
# Check the shape of the immigration table to see how many rows there are 
prov_immigration_df.shape

(980, 5)

In [180]:
# Check the shape of the wages table to see how many rows there are 
wage_df.shape

(312, 3)

In [181]:
# Merge tables on the combination of REF_DATE and province. The combination of these will produce a unique key in either table for a 1-1 relationship.
# The standard behaviour of merge is inner join, which works well for us. We can predict that, with an inner join, we will have as many columns as the smaller of the two tables
# in our new merged table
imm_wage_merge_df = pd.merge(prov_immigration_df, wage_df, on=['REF_DATE','province'])
print(imm_wage_merge_df)

    REF_DATE                   province  Immigrants  Net emigration  \
0       2001                    Alberta     16406.0          6343.0   
1       2001           British Columbia     38484.0         13945.0   
2       2001                   Manitoba      4592.0          1340.0   
3       2001              New Brunswick       798.0           857.0   
4       2001  Newfoundland and Labrador       392.0           465.0   
..       ...                        ...         ...             ...   
307     2024                    Ontario     52781.0          5783.0   
308     2024       Prince Edward Island      1330.0            24.0   
309     2024                     Quebec     13369.0          1544.0   
310     2024               Saskatchewan      5611.0           288.0   
311     2024                      Yukon       297.0            13.0   

     Net non-permanent residents  weekly_wage  
0                         4513.0   714.613068  
1                         9626.0   717.266610  
2  

In [182]:
# Note above that the wage_df.shape statement says we have 312 rows. If our merge worked correctly we should also see that our new merged table has 312 rows, as well
# as having appended the weekly_wage column
print(imm_wage_merge_df)

    REF_DATE                   province  Immigrants  Net emigration  \
0       2001                    Alberta     16406.0          6343.0   
1       2001           British Columbia     38484.0         13945.0   
2       2001                   Manitoba      4592.0          1340.0   
3       2001              New Brunswick       798.0           857.0   
4       2001  Newfoundland and Labrador       392.0           465.0   
..       ...                        ...         ...             ...   
307     2024                    Ontario     52781.0          5783.0   
308     2024       Prince Edward Island      1330.0            24.0   
309     2024                     Quebec     13369.0          1544.0   
310     2024               Saskatchewan      5611.0           288.0   
311     2024                      Yukon       297.0            13.0   

     Net non-permanent residents  weekly_wage  
0                         4513.0   714.613068  
1                         9626.0   717.266610  
2  